# **Set according to environment (e.g. local, Google Colab...)**

In [15]:
project_folder = ''

# **Body**

In [16]:
from custom_libraries.miscellaneous import *
from custom_libraries.image_dataset import *
import numpy as np

In [17]:
import math

def init_weights(masker):

    density = np.count_nonzero(masker == 1) / masker.size
    stddev = math.sqrt(2/(masker.shape[0]*density))
    weights = np.random.normal(size=masker.shape, loc = 0., scale=stddev)
    weights[np.where(masker == 0)] = 0
    weights = np.reshape(weights, [-1])
    weights = weights[np.where(weights != 0)]
    return weights

class treeLayer(tf.keras.layers.Layer):

    def __init__(self, Input_size=3072, use_bias=False, divisor = 2):
        super(treeLayer, self).__init__()
        self.bias = None
        self.summer = None
        self.kernel = None
        self.Input_size = Input_size
        self.use_bias = use_bias
        self.divisor = divisor


    def build(self, input_shape):

        self.summer = np.zeros((self.Input_size, self.Input_size // self.divisor))
        for i in range(self.Input_size):
            self.summer[i, i // self.divisor] = 1

        initializer = init_weights(self.summer)
        initializer = tf.keras.initializers.Constant(initializer)

        self.kernel = self.add_weight(shape=(1, self.Input_size),
                                      initializer=initializer,
                                      trainable=True, dtype=tf.float32)

        self.summer = tf.convert_to_tensor(self.summer, dtype=tf.float32)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.Input_size//self.divisor,),
                                        initializer=tf.keras.initializers.Zeros,
                                        trainable=True)

    def call(self, inputs):
        #print(inputs.shape, self.kernel.shape)
        x = tf.math.multiply(inputs, self.kernel)
        x = tf.matmul(x, self.summer)
        #masked_weights = tf.multiply(self.kernel, self.summer)
        #x = tf.matmul(inputs, masked_weights)
        if self.use_bias:
            x = tf.nn.bias_add(x, self.bias)

        x = tf.nn.leaky_relu(x, alpha = .01)
        return x

def create_model(input_size, num_trees=1, use_bias=False):
    model = tf.keras.Sequential()
    while input_size > num_trees:

        # Check if image is 3-channel
        if input_size % 3 == 0:
            divisor = 3
        else:
            divisor = 2

        model.add(treeLayer(input_size, use_bias=use_bias, divisor=divisor))
        input_size = input_size // divisor

    model.add(tf.keras.layers.Dense(units=1, activation='sigmoid', use_bias=use_bias))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, epsilon=1e-08),
                  loss=tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.AUTO),
                  metrics=['binary_crossentropy', 'acc'])

    return model


In [18]:
import gc

# Initialize settings
bs = 256
trials = 10
epochs = 2000
trees_set = [1]

# Load class-dataset list
# classes = np.load(project_folder + 'results/classes.npy', allow_pickle=True)

classes = [[5, 6, 'svhn']]

callback = tf.keras.callbacks.EarlyStopping(monitor='val_binary_crossentropy', patience=60)

#history = np.load(project_folder + 'results/fcnn_history.npy', allow_pickle=True)
history = np.zeros((len(classes), trials, len(trees_set), 2))

# For each dataset enumerated from classes list
for j, (t1, t2, ds) in enumerate(classes):

    print(f"Dataset: {ds} / Pair: {t1}-{t2}")

    test_ds = ImageDataset(ds, 'test', data_dir=None)
    train_ds = ImageDataset(ds, 'train', data_dir=None)

    for x in [train_ds, test_ds]:
        x.filter(t1, t2, overwrite=True)
        x.shuffle()
        x.normalize()
        if x.images.shape[1:3] == (28, 28):
            x.pad()
        x.vectorize(True)

    for k, trees in enumerate(trees_set):

        print(f"{trees}-FCNN")
        X_train, y_train, X_valid, y_valid = train_ds.subset(shard=True, shard_number=trials, validation=True,
                                                             validation_size=len(test_ds.images))
        X_test, y_test = test_ds.subset()
        test_set = tf.data.Dataset.from_tensor_slices((X_test, y_test)).map(
            lambda x, y: (tf.tile(x, [trees]), y)).batch(bs)

        #if history[j, -1, k, 0] != 0:
        #  continue

        for i in range(trials):
            print(f"Trial {i + 1}")

            #with tf.device('/device:GPU:0'):

            model = create_model(input_size=X_train[i].shape[1] * trees, num_trees=trees, use_bias=False)

            train_set = tf.data.Dataset.from_tensor_slices((X_train[i], y_train[i])).map(
                lambda x, y: (tf.tile(x, [trees]), y)).batch(bs)
            valid_set = tf.data.Dataset.from_tensor_slices((X_valid[i], y_valid[i])).map(
                lambda x, y: (tf.tile(x, [trees]), y)).batch(bs)

            fit_history = model.fit(x=train_set, batch_size=bs, epochs=epochs,
                                    validation_data=valid_set, validation_batch_size=bs,
                                    callbacks=[callback], verbose=0)
            print_fit_history(fit_history, epochs)

            evaluate_history = model.evaluate(x=test_set, batch_size=bs, verbose=0)
            print_evaluate_history(evaluate_history)

            history[j, i, k] = evaluate_history[1:]

            #np.save(project_folder + 'results/ktree_history.npy', history,
            #        allow_pickle=True)

            del model, train_set, valid_set
            gc.collect()

Dataset: svhn / Pair: 5-6
1-FCNN
Trial 1
Epochs: 105/2000 - Train BCE: 0.6929, accuracy: 47.03% - Validation BCE: 0.6934, accuracy: 40.96%
Test BCE: 0.6929, accuracy: 47.49%
Trial 2
Epochs: 422/2000 - Train BCE: 0.6257, accuracy: 59.52% - Validation BCE: 0.6646, accuracy: 57.34%
Test BCE: 0.6704, accuracy: 55.26%
Trial 3
Epochs: 301/2000 - Train BCE: 0.6307, accuracy: 67.64% - Validation BCE: 0.6818, accuracy: 60.55%
Test BCE: 0.7146, accuracy: 61.04%
Trial 4
Epochs: 62/2000 - Train BCE: 0.6931, accuracy: 54.79% - Validation BCE: 0.6931, accuracy: 53.9%
Test BCE: 0.6931, accuracy: 54.74%
Trial 5
Epochs: 61/2000 - Train BCE: 0.6931, accuracy: 53.45% - Validation BCE: 0.6932, accuracy: 55.05%
Test BCE: 0.6931, accuracy: 54.09%
Trial 6
Epochs: 150/2000 - Train BCE: 0.6931, accuracy: 47.15% - Validation BCE: 0.6931, accuracy: 42.2%
Test BCE: 0.6931, accuracy: 43.57%
Trial 7
Epochs: 565/2000 - Train BCE: 0.6342, accuracy: 68.0% - Validation BCE: 0.6698, accuracy: 59.86%
Test BCE: 0.6807, ac

In [19]:
history = np.load(project_folder + 'results/ktree_history.npy', allow_pickle=True)
print("RESULTS:")
for j, (t1, t2, ds) in enumerate(classes):
    print(f"Dataset: {ds} / Pair: {t1}-{t2}")
    for k, trees in enumerate(trees_set):
        print(f"{trees}-tree")
        print(f"Accuracy: mean = {round(np.mean(history[j,:,k,1]), 4)}, standard deviation = {round(np.std(history[j,:,k,1]), 4)}")

FileNotFoundError: [Errno 2] No such file or directory: 'results/ktree_history.npy'